In [ ]:
import operator
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

# 1. Define State
class GraphState(TypedDict):
    messages: Annotated[List[str], operator.add]

# 2. Define Nodes
def scalar_model_node(state: GraphState):
    return {"messages": ["Scalar Output"]}

def gemini_node(state: GraphState):
    return {"messages": ["Gemini Result"]}

def claude_node(state: GraphState):
    return {"messages": ["Claude Result"]}

def consolidation_node(state: GraphState):
    print("--> Consolidation: Merging results...")
    return {"messages": ["Consolidated Final Answer"]}

def email_results_node(state: GraphState):
    print("--> Emailing Results: Sending data...")
    return {}

# 3. Build Graph
builder = StateGraph(GraphState)
builder.add_node("scalar_model", scalar_model_node)
builder.add_node("gemini", gemini_node)
builder.add_node("claude", claude_node)
builder.add_node("consolidation", consolidation_node)
builder.add_node("email_results", email_results_node)

builder.add_edge(START, "scalar_model")
builder.add_edge("scalar_model", "gemini")
builder.add_edge("scalar_model", "claude")
builder.add_edge("gemini", "consolidation")
builder.add_edge("claude", "consolidation")
builder.add_edge("consolidation", "email_results")
builder.add_edge("email_results", END)

# 4. Compile with Checkpointer
memory = MemorySaver()
graph = builder.compile(checkpointer=memory, interrupt_before=["email_results"])

# Visualize
display(Image(graph.get_graph().draw_mermaid_png()))
